In [1]:
import sys
from pathlib import Path

# change to your NFF path
sys.path.insert(0, "..")
sys.path.insert(0, "../..")
sys.path.insert(0, "../../../")

import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import copy

import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler

from nff.data import Dataset, split_train_validation_test, collate_dicts, to_tensor
from nff.train import Trainer, get_trainer, get_model, load_model, loss, hooks, metrics, evaluate

import argparse
from sigopt import Connection

from train import train
from forceconv import *

from MD17data import *

from forcepai import ForcePai
# from nff.nn.models import Painn
from forcedime import ForceDime

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("-logdir", type=str, default='./output')
parser.add_argument("-device", type=int, default=0)
parser.add_argument("-data", type=str, default='ethanol_dft')
params = vars(parser.parse_args([]))

DEVICE = params['device']
OUTDIR = '{}/{}/sandbox'.format(params['logdir'], 'test_ForcePai')

BATCH_SIZE = 10
lr = 1e-5
n_epochs = 100

In [3]:
data = get_MD17data(params['data'])
dataset = pack_MD17data(data, 10000)
dataset.generate_angle_list()
dataset.generate_kj_ji()

100%|██████████| 10000/10000 [00:34<00:00, 292.87it/s]


Adding kj and ji indices with 1 parallel processes


100%|██████████| 10000/10000 [00:03<00:00, 2593.43it/s]


In [4]:
train, val, test = split_train_validation_test(dataset, val_size=0.05, test_size=0.85)
train_loader = DataLoader(train, batch_size=BATCH_SIZE, collate_fn=collate_dicts)
val_loader = DataLoader(val, batch_size=BATCH_SIZE, collate_fn=collate_dicts)
test_loader = DataLoader(test, batch_size=BATCH_SIZE, collate_fn=collate_dicts)

In [5]:
# modelparams = {"feat_dim": 128,
#               "activation": "swish",
#               "n_rbf": 20,
#               "cutoff": 5.0,
#               "num_conv": 3,
#               "output_keys": ["energy"],
#               "grad_keys": ["energy_grad"],
#                # whether to sum outputs from all blocks in the model
#                # or just the final output block. False in the original
#                # implementation
#               "skip_connection": False,
#                # Whether the k parameters in the Bessel basis functions
#                # are learnable. False originally
#               "learnable_k": False,
#                # dropout rate in the convolution layers, originally 0
#                "conv_dropout": 0.0,
#                # dropout rate in the readout layers, originally 0
#                "readout_dropout": 0.0,
#                # dictionary of means to add to each output key
#                # (this is optional - if you don't supply it then
#                # nothing will be added)
#                # "means": {"energy": train.props['energy'].mean().item()},
#                # dictionary of standard devations with which to 
#                # multiply each output key
#                # (this is optional - if you don't supply it then
#                # nothing will be multiplied)
#                # "stddevs": {"energy": train.props['energy'].std().item()}
#               }
# model = ForcePai(modelparams).to(DEVICE)
model = ForceDime(n_rbf=16, cutoff=5, 
                envelope_p=8, l_spher=6, 
                n_spher=6, embed_dim=128, 
                activation='ReLU', 
                n_bilinear=8, 
                n_convolutions=6).to(DEVICE)

In [6]:
loss_fn = loss.build_mse_loss(loss_coef={'energy_grad': 1})
trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = Adam(trainable_params, lr=lr)
train_metrics = [
        metrics.MeanAbsoluteError('energy_grad')
    ]

In [ ]:
train_hooks = [
    hooks.MaxEpochHook(n_epochs),
    hooks.CSVHook(
        OUTDIR,
        metrics=train_metrics,
    ),
    hooks.PrintingHook(
        OUTDIR,
        metrics=train_metrics,
        separator = ' | ',
        time_strf='%M:%S'
    ),
    hooks.ReduceLROnPlateauHook(
        optimizer=optimizer,
        patience=20,
        factor=0.5,
        min_lr=1e-7,
        window_length=1,
        stop_after_min=True
    )
]

T = Trainer(
    model_path=OUTDIR,
    model=model,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_loader=train_loader,
    validation_loader=val_loader,
    checkpoint_interval=1,
    hooks=train_hooks,
    mini_batches=1
)

T.train(device=DEVICE, n_epochs=1000)

In [5]:
from nff.utils.cuda import batch_to, batch_detach
data = None
for batch in train_loader:
    data = batch_to(batch, DEVICE)
    break

In [11]:
data['angle_list'][:, 0:2]==data['nbr_list']

RuntimeError: The size of tensor a (5040) must match the size of tensor b (720) at non-singleton dimension 0

In [16]:
data['kj_idx']

tensor([  0,   0,   0,  ..., 719, 719, 719], device='cuda:0')

In [7]:
rotate = torch.Tensor([[2**.5/2, -2**.5/2, 0],
                       [2**.5/2, 2**.5/2, 0],
                       [0, 0, 1]]).to(DEVICE)
rotate

tensor([[ 0.7071, -0.7071,  0.0000],
        [ 0.7071,  0.7071,  0.0000],
        [ 0.0000,  0.0000,  1.0000]], device='cuda:0')

In [13]:
data['nxyz'][:, 1:4] @= rotate

In [8]:
# results = batch_to(batch_detach(model(data)), DEVICE)
results = model(data)

In [9]:
results

(tensor([[ -19.0282,   12.1743,    6.9539,  ...,  -68.2679, -118.7275,
            27.2735],
         [ -33.1151,    8.4410,  -17.9287,  ...,  -31.6520,  -78.1353,
             5.2346],
         [ -24.3289,   87.1462,   32.8571,  ...,  -28.7307,  -15.7856,
            41.0139],
         ...,
         [  25.9720,   56.2161,  -19.8533,  ...,  -18.3722,  -34.1797,
           -37.5298],
         [  10.0748,   62.3371,   11.9721,  ...,  -68.0998,  -44.8047,
            20.4965],
         [  21.2208,   34.7819,    8.4771,  ...,  -34.2247,  -13.6852,
            17.6991]], device='cuda:0', grad_fn=<AddBackward0>),
 tensor([[  664.2386,  -889.8787,   391.8072,  ...,   184.7081,  -333.9512,
           -150.7593],
         [  612.2758, -1044.0516,   497.1471,  ...,   331.9441,  -438.3710,
           -195.1217],
         [  725.1776,  -952.8761,   359.3198,  ...,    68.3555,  -212.0107,
           -186.9438],
         ...,
         [  108.9786,  -575.1990,    50.4693,  ...,   -11.4498,   178.5034

In [14]:
new_results = model(data)

In [14]:
(new_results['energy_grad'] - (results['energy_grad'] @ rotate)).mean()

tensor(-4.5393e-09, device='cuda:0')

In [15]:
new_results

(tensor([[ -19.0282,   12.1743,    6.9539,  ...,  -68.2679, -118.7275,
            27.2735],
         [ -33.1151,    8.4410,  -17.9287,  ...,  -31.6520,  -78.1353,
             5.2346],
         [ -24.3289,   87.1463,   32.8571,  ...,  -28.7307,  -15.7856,
            41.0139],
         ...,
         [  25.9720,   56.2161,  -19.8533,  ...,  -18.3722,  -34.1797,
           -37.5298],
         [  10.0748,   62.3371,   11.9721,  ...,  -68.0998,  -44.8047,
            20.4965],
         [  21.2208,   34.7818,    8.4770,  ...,  -34.2246,  -13.6851,
            17.6991]], device='cuda:0', grad_fn=<AddBackward0>),
 tensor([[  664.2389,  -889.8787,   391.8072,  ...,   184.7081,  -333.9511,
           -150.7593],
         [  612.2759, -1044.0515,   497.1472,  ...,   331.9442,  -438.3709,
           -195.1216],
         [  725.1777,  -952.8764,   359.3197,  ...,    68.3552,  -212.0107,
           -186.9438],
         ...,
         [  108.9790,  -575.1990,    50.4692,  ...,   -11.4500,   178.5031

In [39]:
from nff.utils.tools import make_directed
nbrs, _ = make_directed(batch['nbr_list'])
nxyz = data['nxyz']
xyz = nxyz[:, 1:]
xyz.requires_grad = True

def norm(vec):
    EPS = 1e-15
    result = ((vec ** 2 + EPS).sum(-1)) ** 0.5
    return result

def func(xyz):
    r_ij = xyz[nbrs[:, 1]] - xyz[nbrs[:, 0]]
    dist = norm(r_ij)
    unit = r_ij / dist.reshape(-1, 1)
    return unit

In [40]:
a = torch.autograd.functional.jacobian(func, xyz)

In [49]:
a.shape

torch.Size([720, 3, 90, 3])

In [50]:
a

tensor([[[[-0.6741,  0.0282,  0.0122],
          [ 0.6741, -0.0282, -0.0122],
          [ 0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],

         [[ 0.0282, -0.1079,  0.2459],
          [-0.0282,  0.1079, -0.2459],
          [ 0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],

         [[ 0.0122,  0.2459, -0.5690],
          [-0.0122, -0.2459,  0.5690],
          [ 0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]]],


        [[[-0.6496, -0.1210,  0.1654],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.6496,  0.1210, -0.1654],
          ...,
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.000

In [44]:
r_ij = xyz[nbrs[:, 1]] - xyz[nbrs[:, 0]]
dis_vec = r_ij
dis = norm(r_ij).unsqueeze(-1)
dis_adjoint = dis_vec / dis  # N_e * 3
dis_order2 = dis.unsqueeze(-1)
eye3 = torch.eye(3).unsqueeze(0).to(dis)
b = ((-dis_order2*eye3) - (r_ij.unsqueeze(-2)*r_ij.unsqueeze(-1))/dis_order2) / dis_order2**2

In [45]:
b.shape

torch.Size([720, 3, 3])

In [46]:
xyz.shape

torch.Size([90, 3])

In [47]:
from nff.utils.scatter import scatter_add
scatter_add(a, nbrs[:,0], dim=2, dim_size=90) 

RuntimeError: shape '[1, 1, 90, 1]' is invalid for input of size 720